# 01 - Data Exploration: PianoVAM Dataset

This notebook explores the PianoVAM dataset for piano fingering detection.

**Contents:**
- Load dataset from HuggingFace
- Visualize sample videos and frames
- Parse and visualize MIDI data
- Explore hand skeleton annotations
- Understand data quality and characteristics


In [ ]:
# Install dependencies (run once in Colab)
# !pip install -q datasets huggingface_hub opencv-python mido matplotlib seaborn tqdm

import os
import sys

# Setup for Colab
if 'google.colab' in str(get_ipython()):
    if not os.path.exists('computer-vision'):
        !git clone https://github.com/esnylmz/computer-vision.git
    os.chdir('computer-vision')
    !pip install -q -e .
else:
    # Local development
    sys.path.insert(0, '..')

# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from tqdm.notebook import tqdm

print("Setup complete!")


## 1. Load PianoVAM Dataset

Load the dataset from HuggingFace and explore its structure.


In [ ]:
# Import project modules
from src.data.dataset import PianoVAMDataset
from src.data.midi_utils import MidiProcessor
from src.data.video_utils import VideoProcessor
from src.utils.config import load_config

# Load configuration
config = load_config('configs/default.yaml')
print(f"Project: {config.project_name} v{config.version}")

# =============================================================================
# PianoVAM Dataset Split Names - IMPORTANT!
# =============================================================================
# HuggingFace API split names: 'train', 'validation', 'test'
#   - train: 73 samples
#   - validation: 19 samples  
#   - test: 14 samples
#
# NOTE: The 'split' COLUMN inside each sample has DIFFERENT values:
#   - 'train', 'valid', 'test' (note: 'valid' NOT 'validation')
#
# Our loader accepts: 'train', 'validation', 'valid', 'val', 'test'
#   - 'valid' and 'val' are automatically mapped to 'validation'
# =============================================================================

print("\nLoading PianoVAM dataset with streaming mode...")
print("Using streaming=True to avoid downloading all 432 files upfront")
print("Limiting to 20 samples for quick exploration...\n")

# Load train split (streaming mode for efficiency)
train_dataset = PianoVAMDataset(split='train', streaming=True, max_samples=20)
print("Train dataset loaded (first 20 samples)\n")

# Load validation split - use 'validation' (API name) or 'valid'/'val' (auto-mapped)
val_dataset = PianoVAMDataset(split='validation', streaming=True, max_samples=20)
print("Validation dataset loaded (first 20 samples)\n")

# Load test split
test_dataset = PianoVAMDataset(split='test', streaming=True, max_samples=20)
print("Test dataset loaded (first 20 samples)")

print("\n" + "="*60)
print("Dataset loading complete!")
print("="*60)
print("\nUsage tips:")
print("  - streaming=True: Fast, but can only iterate (not index)")
print("  - streaming=False: Allows indexing like dataset[0], but slower to load")
print("  - max_samples=N: Limit samples for quick exploration")


In [ ]:
# Explore a sample (using iteration for streaming mode)
sample = next(iter(train_dataset))

print(f"Sample ID: {sample.id}")
print(f"Composer: {sample.metadata['composer']}")
print(f"Piece: {sample.metadata['piece']}")
print(f"Performer: {sample.metadata['performer']}")
print(f"Skill Level: {sample.metadata['skill_level']}")
print(f"Duration: {sample.metadata['duration']:.1f}s")
print(f"\nKeyboard Corners: {sample.metadata['keyboard_corners']}")
print(f"\nPaths:")
print(f"  Video: {sample.video_path[:80]}...")
print(f"  MIDI: {sample.midi_path[:80]}...")
print(f"  Skeleton: {sample.skeleton_path[:80]}...")
